In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
from pathlib import Path

from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

from nerfstudio.utils.eval_utils import eval_setup
# from ns_extension.utils.grouping import GroupingClassifier
import collab_splats

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Taichi] version 1.7.4, llvm 15.0.4, commit b4b956fd, linux, python 3.10.18


[I 09/07/25 18:32:28.439 5310] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


### Load configuration

In [2]:
# Path to the config for a trained model
load_config = '/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-25_074037/config.yml'
# load_config = '/workspace/fieldwork-data/birds/2024-02-06/environment/C0043/rade-features/2025-07-25_040743/config.yml'
load_config = Path(load_config)

# config, pipeline, checkpoint_path, step = eval_setup(load_config)

### Question is whether to build grouping on top of the existing data?

In [3]:
from PIL import Image
import numpy as np

from collab_splats.utils.features import resize_image
from collab_splats.utils.segmentation import Segmentation

segmentation = Segmentation(
    backend='mobilesamv2',
    strategy='object',
    device='cuda',
)
segmentation.strategy = 'auto'

Using cache found in /workspace/models/hub/RogerQi_MobileSAMV2_main


checkpoint_load_scucess


### Load a classifier

In [3]:
from collab_splats.utils.grouping import GroupingClassifier, GroupingConfig

# Path to the config for a trained model
load_config = '/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-25_074037/config.yml'
load_config = Path(load_config)

grouping_config = GroupingConfig(segmentation_backend='mobilesamv2', segmentation_strategy='object', front_percentage=0.2, iou_threshold=0.1, num_patches=32)
grouping_classifier = GroupingClassifier(load_config=load_config, config=grouping_config)

Memory bank loaded from /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/grouping/memory_bank.pkl with 548 masks


### Associate masks

In [ ]:
# Create the masks for each image
grouping_classifier.create_masks()

In [ ]:
# Associate the masks across frames
grouping_classifier.associate()

Loading train ordered data...


Processing frames:  49%|████▉     | 217/441 [00:03<00:03, 59.63it/s]

No objects found in frame_00232.png
No objects found in frame_00235.png


Processing frames: 100%|██████████| 441/441 [00:07<00:00, 58.62it/s]


### Train classifier 

In [ ]:
grouping_classifier.setup_train()

In [ ]:
for i in range(10):

    batch = grouping_classifier.pipeline.datamanager.next_train(i)
    break

In [ ]:
# dataset = grouping_classifier.pipeline.datamanager.train_dataset

for x in tqdm(grouping_classifier.pipeline.datamanager.train_dataset):
    print (x)
    break

In [ ]:
grouping_classifier.pipeline.datamanager.next_train()

In [ ]:
for x in grouping_classifier.pipeline.:
    print(x)
    break

In [ ]:
grouping_classifier.train_classifier()